<a href="https://colab.research.google.com/github/minat-hub/Machine-learning-projects/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     |████████████████████████████████| 399.9MB 41kB/s 
     |████████████████████████████████| 12.2MB 25.5MB/s 
     |████████████████████████████████| 471kB 41.6MB/s 
     |████████████████████████████████| 4.0MB 45.9MB/s 
     |████████████████████████████████| 4.0MB 45.7MB/s 
ERROR: tensorflow 2.4.0 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
2.5.0-dev20210108


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-01-08 11:28:17--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv          [ <=>                ] 349.84K  --.-KB/s    in 0.02s   

2021-01-08 11:28:18 (14.8 MB/s) - ‘train-data.tsv’ saved [358233]

--2021-01-08 11:28:18--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv          [ <=>                ] 115

In [3]:
# Load training data into dataframe
train_dataframe = pd.read_csv(train_file_path,delimiter='\t',encoding='utf-8',names=["HamOrSpam","sms"])

In [4]:
train_dataframe['HamOrSpam'].value_counts()
train_dataframe.tail

<bound method NDFrame.tail of      HamOrSpam                                                sms
0          ham  ahhhh...just woken up!had a bad dream about u ...
1          ham                           you can never do nothing
2          ham  now u sound like manky scouse boy steve,like! ...
3          ham  mum say we wan to go then go... then she can s...
4          ham  never y lei... i v lazy... got wat? dat day ü ...
...        ...                                                ...
4174       ham  just woke up. yeesh its late. but i didn't fal...
4175       ham  what do u reckon as need 2 arrange transport i...
4176      spam  free entry into our £250 weekly competition ju...
4177      spam  -pls stop bootydelious (32/f) is inviting you ...
4178       ham  tell my  bad character which u dnt lik in me. ...

[4179 rows x 2 columns]>

In [5]:
train_sms = train_dataframe['sms']
train_HamOrSpam = train_dataframe['HamOrSpam']

In [6]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)

In [7]:
tokenize.fit_on_texts(train_sms) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_sms)

In [8]:
# Using sklearn utility to convert label strings to numbered index
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

encoder = LabelEncoder()
encoder.fit(train_HamOrSpam)
y_train = encoder.transform(train_HamOrSpam)


In [9]:
# This converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
# Inspecting the dimenstions of our training and test data (this is helpful for debugging)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

x_train shape: (4179, 1000)
y_train shape: (4179, 2)


In [10]:
# Training for more epochs will likely lead to overfitting on this dataset
batch_size = 32
epochs = 2
# drop_ratio = 0.5

In [11]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(512, input_shape=(max_words,)))
model.add(tf.keras.layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(tf.keras.layers.Dense(num_classes))
model.add(tf.keras.layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/2
118/118 [==============================] - 1s 9ms/step - loss: 0.3272 - accuracy: 0.8600 - val_loss: 0.0480 - val_accuracy: 0.9904
Epoch 2/2
118/118 [==============================] - 1s 4ms/step - loss: 0.0299 - accuracy: 0.9948 - val_loss: 0.0350 - val_accuracy: 0.9904


In [13]:
test_dataframe = pd.read_csv(test_file_path,delimiter='\t',encoding='utf-8',names=["HamOrSpam","sms"])
test_sms = test_dataframe['sms']
test_HamOrSpam = test_dataframe['HamOrSpam']
x_test = tokenize.texts_to_matrix(test_sms)
y_test = encoder.transform(test_HamOrSpam)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

x_test shape: (1392, 1000)
y_test shape: (1392, 2)


In [14]:
score = model.evaluate(x_test,y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

44/44 [==============================] - 0s 2ms/step - loss: 0.0446 - accuracy: 0.9878
Test loss: 0.04463491588830948
Test accuracy: 0.9877873659133911


In [15]:
def predict_message(pred_text):
    hamOrspam = ['ham','spam']
    prediction = model.predict(tokenize.texts_to_matrix([pred_text]))
    predicted_position = np.argmax(prediction)
    

    return ([prediction[0][predicted_position],hamOrspam[predicted_position]])

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)

[0.9973546, 'ham']


In [16]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    hamOrspam = ['ham','spam']
    prediction = model.predict(tokenize.texts_to_matrix([pred_text]))
    predicted_position = np.argmax(prediction)
    

    return ([prediction[0][predicted_position],hamOrspam[predicted_position]])

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.9973546, 'ham']


In [17]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
